In [4]:
# https://www.tensorflow.org/tutorials/keras/regression

from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import os

In [5]:
# Import helpers
import import_ipynb
from common import buildDataset
from common import filterBad
from common import splitDataset
from common import worstOffenders
from common import dropColumns
from common import normDataset
from common import evaluatePerf
from common import evaluateCustom
from common import extractNNWeights

In [6]:
only_use_strong = 54

dataset, columns = buildDataset()
dataset = filterBad(dataset, only_use_strong)
# dataset = dropColumns(dataset, ['fileId'])
train_dataset, test_dataset, train_labels, test_labels = splitDataset(dataset, 0.1)
train_stats = train_dataset.describe().transpose()

# FCNN-specific
normed_train_data = normDataset(train_dataset, train_stats)
normed_test_data = normDataset(test_dataset, train_stats)

# print(train_dataset)
# print(normed_train_data)

Rejected 7436440 points (68.220402%)


In [4]:
# hyperparameters
scale = 128.0 # divide rising/falling idx by this number so it fits in fpga precision


In [5]:
def train_model(x,y,degree):
  model = np.polyfit(x,y,degree)
  return model

def predict(pt, rising_model, falling_model):
    tyr = np.sum([m * math.pow(pt['rising_idx'] / scale,i) for i,m in enumerate(rising_model[::-1])])
    tyf = np.sum([m * math.pow(pt['falling_idx'] / scale,i) for i,m in enumerate(falling_model[::-1])])
    
    if (pt['first_val'] >= 30): return tyf # only use falling
    elif (pt['last_val'] >= 30): return tyr # only use rising
    return (tyr + tyf) / 2 # use both rising and falling


In [6]:
def trainAndBuild(scale, degree):
    rising_dataset = train_dataset[(train_dataset['first_val'] < 30)]
    rising_labels = train_labels[(train_dataset['first_val'] < 30)]
    rising_model = train_model(rising_dataset['rising_idx'] / scale, rising_labels, degree)
#     print('val rising_model = Seq(%s)' % ','.join([str(x) + '.to[T]' for x in rising_model]))
    falling_dataset = train_dataset[(train_dataset['last_val'] < 30)]
    falling_labels = train_labels[(train_dataset['last_val'] < 30)]
    falling_model = train_model(falling_dataset['falling_idx'] / scale, falling_labels, degree)
#     print('val falling_model = Seq(%s)' % ','.join([str(x) + '.to[T]' for x in falling_model]))
    print('  def Polyfit_%d_params[T](cast: scala.Double => T) = (Seq(%s).map(cast), Seq(%s).map(cast))' % (degree, ','.join([str(x) for x in rising_model]), ','.join([str(x) for x in falling_model])))
    return rising_model, falling_model

# Model playground

In [9]:
# polyfit_2
rising_model_polyfit_2, falling_model_polyfit_2 = trainAndBuild(scale, 2)

# polyfit_3
rising_model_polyfit_3, falling_model_polyfit_3 = trainAndBuild(scale, 3)

# polyfit_4
rising_model_polyfit_4, falling_model_polyfit_4 = trainAndBuild(scale, 4)

# Evaluations
preds = [predict(x[1], rising_model_polyfit_2, falling_model_polyfit_2) for x,y in zip(test_dataset.iterrows(), test_labels)]
evaluatePerf(test_labels, preds)
preds = [predict(x[1], rising_model_polyfit_3, falling_model_polyfit_3) for x,y in zip(test_dataset.iterrows(), test_labels)]
evaluatePerf(test_labels, preds)
preds = [predict(x[1], rising_model_polyfit_4, falling_model_polyfit_4) for x,y in zip(test_dataset.iterrows(), test_labels)]
evaluatePerf(test_labels, preds)

NameError: name 'trainAndBuild' is not defined

In [20]:
def worstOffenders(y, preds, data, labels):
    import numpy as np
    mae = np.abs(y - preds)
    mse = (y - preds) * (y - preds)
    mae_mean, mae_std = np.mean(mae), np.std(mae)
    mse_mean, mse_std = np.mean(mse), np.std(mse)
    print('mae mean: %.3f, stddev: %.3f' % (mae_mean, mae_std))
    print('mse mean: %.3f, stddev: %.3f' % (mse_mean, mse_std))
    print('MAE Outliers:')
    for i,x in enumerate(mae):
        if ((x - mae_mean) > 3 * mae_std):
            print('Error: %.1f (want %.1f, got %.1f), file %d, row %d, _idx %d %d, _val %d %d, volume %.0f' % (
                                                                                            y.iloc[i] - preds[i],
                                                                                            y.iloc[i], preds[i],
                                                                                            data.iloc[i]['fileId'],
                                                                                            data.iloc[i]['row'],
                                                                                            data.iloc[i]['rising_idx'],
                                                                                            data.iloc[i]['falling_idx'],
                                                                                            data.iloc[i]['first_val'],
                                                                                            data.iloc[i]['last_val'],
                                                                                            data.iloc[i]['volume']))
worstOffenders(test_labels, preds, test_dataset, test_labels)

mae mean: 24.027, stddev: 42.339
mse mean: 2369.910, stddev: 88488.659
MAE Outliers:
Error: 214.3 (want -1783.8, got -1998.1), file 98107, row 15, _idx 450 766, _val 21542 0, volume 16644053
Error: 253.8 (want -475.1, got -728.8), file 20713, row 11, _idx 291 530, _val 65535 0, volume 11970949
Error: 189.8 (want 785.3, got 595.5), file 48147, row 19, _idx 142 326, _val 20771 0, volume 8358140
Error: 277.0 (want 893.8, got 616.8), file 67157, row 36, _idx 101 323, _val 65535 0, volume 11447764
Error: 204.7 (want -635.9, got -840.7), file 45872, row 39, _idx 333 549, _val 28546 0, volume 11155543
Error: 201.9 (want -1987.4, got -2189.4), file 83487, row 45, _idx 471 806, _val 14425 0, volume 18260501
Error: 178.5 (want 1371.8, got 1193.3), file 70494, row 51, _idx 102 215, _val 0 0, volume 102
Error: 155.4 (want -1409.0, got -1564.5), file 60359, row 4, _idx 380 680, _val 7718 0, volume 16420885
Error: 175.2 (want -1336.8, got -1512.0), file 9712, row 6, _idx 371 670, _val 9961 0, volume

Error: 155.5 (want -2419.5, got -2575.0), file 16640, row 22, _idx 542 891, _val 6949 0, volume 19357621
Error: 221.2 (want 845.2, got 623.9), file 56096, row 49, _idx 98 322, _val 30955 0, volume 11113312
Error: 173.8 (want -788.4, got -962.2), file 39227, row 16, _idx 292 570, _val 11023 0, volume 15082248
Error: 170.5 (want 916.4, got 745.9), file 44991, row 31, _idx 84 305, _val 9538 0, volume 10877209
Error: 156.8 (want 1612.1, got 1455.3), file 43841, row 37, _idx 40 211, _val 53567 0, volume 8026859
Error: 239.9 (want -1340.2, got -1580.1), file 8911, row 3, _idx 381 683, _val 48962 0, volume 16522902
Error: 178.9 (want 407.2, got 228.3), file 4059, row 14, _idx 142 379, _val 11981 0, volume 12030625
Error: 181.5 (want -100.3, got -281.8), file 25986, row 13, _idx 206 457, _val 14520 0, volume 12988560
Error: 195.8 (want 498.8, got 303.1), file 3173, row 1, _idx 140 368, _val 34317 0, volume 12080960
Error: 303.2 (want -531.6, got -834.8), file 23024, row 10, _idx 334 548, _val 

Error: 215.9 (want -1777.3, got -1993.2), file 28845, row 27, _idx 450 765, _val 20760 0, volume 16275658
Error: 182.6 (want 424.4, got 241.8), file 23350, row 25, _idx 141 377, _val 15969 0, volume 12114622
Error: 226.2 (want -74.5, got -300.7), file 185, row 17, _idx 200 460, _val 39743 0, volume 13422310
Error: 201.4 (want 990.8, got 789.4), file 33820, row 31, _idx 80 299, _val 28025 0, volume 11127037
Error: 239.6 (want -1850.1, got -2089.7), file 80417, row 26, _idx 465 785, _val 40059 0, volume 17381038
Error: 231.1 (want 713.9, got 482.8), file 66751, row 14, _idx 111 342, _val 65535 0, volume 12155788
Error: -196.9 (want -272.1, got -75.2), file 66654, row 47, _idx 256 391, _val 0 0, volume 674
Error: 180.1 (want 1006.0, got 825.8), file 39346, row 24, _idx 75 294, _val 14526 0, volume 10727832
Error: 219.9 (want -532.6, got -752.5), file 74563, row 26, _idx 332 534, _val 50874 0, volume 10428588
Error: 152.7 (want 642.5, got 489.8), file 35506, row 12, _idx 142 341, _val 7944

Error: 161.6 (want 573.5, got 411.9), file 89586, row 42, _idx 194 320, _val 0 0, volume 107
Error: 283.4 (want 1022.0, got 738.7), file 72856, row 5, _idx 86 306, _val 65535 0, volume 10747577
Error: 166.2 (want -2116.4, got -2282.6), file 83509, row 41, _idx 525 826, _val 9022 0, volume 16535787
Error: 195.7 (want -297.8, got -493.5), file 73036, row 36, _idx 251 491, _val 19020 0, volume 12505247
Error: -188.7 (want -1299.0, got -1110.3), file 47330, row 39, _idx 401 559, _val 0 0, volume 280
Error: 292.8 (want 916.8, got 623.9), file 58336, row 5, _idx 141 322, _val 65535 0, volume 8074164
Error: -191.1 (want 275.0, got 466.1), file 83295, row 17, _idx 185 315, _val 0 0, volume 2971
Error: 233.4 (want -136.3, got -369.7), file 27412, row 33, _idx 209 471, _val 41031 0, volume 13691731
Error: -180.9 (want -510.4, got -329.5), file 58662, row 28, _idx 289 431, _val 0 0, volume 765
Error: 199.4 (want -324.8, got -524.1), file 46768, row 3, _idx 268 496, _val 18319 0, volume 10891213
E

Error: 295.2 (want -24.5, got -319.6), file 50201, row 14, _idx 193 463, _val 65535 0, volume 14588566
Error: 185.6 (want 1048.1, got 862.4), file 70756, row 5, _idx 86 289, _val 21090 0, volume 9630424
Error: 3363.0 (want 413.1, got -2949.8), file 51888, row 26, _idx 144 980, _val 65535 0, volume 14641979
Error: 237.1 (want -238.0, got -475.1), file 19659, row 46, _idx 120 488, _val 59965 0, volume 18664710
Error: -186.3 (want 547.3, got 733.6), file 17380, row 37, _idx 152 279, _val 0 0, volume 1007
Error: 225.2 (want -2011.0, got -2236.2), file 42614, row 9, _idx 552 816, _val 30550 0, volume 13966223
Error: 215.5 (want -302.6, got -518.0), file 73395, row 29, _idx 229 495, _val 30472 0, volume 14025576
Error: 163.1 (want 923.5, got 760.4), file 52785, row 18, _idx 83 303, _val 11370 0, volume 11206826
Error: 182.3 (want -486.9, got -669.3), file 89536, row 38, _idx 276 520, _val 17246 0, volume 12679570
Error: 179.6 (want 537.5, got 357.9), file 53120, row 7, _idx 131 360, _val 160

...

Error: 224.5 (want -1615.9, got -1840.4), file 54465, row 52, _idx 454 734, _val 31554 0, volume 15087134
Error: 170.5 (want -1885.6, got -2056.1), file 91308, row 28, _idx 454 778, _val 7619 0, volume 17477308
Error: -188.6 (want -252.4, got -63.9), file 92702, row 52, _idx 253 391, _val 0 0, volume 1894
Error: 169.6 (want 250.5, got 80.9), file 35935, row 6, _idx 167 401, _val 12988 0, volume 11992482
Error: 168.1 (want -1453.6, got -1621.7), file 96847, row 4, _idx 388 691, _val 7929 0, volume 16756841
Error: 210.3 (want 763.4, got 553.1), file 6401, row 18, _idx 108 332, _val 30845 0, volume 11156641
Error: 225.2 (want -1802.0, got -2027.2), file 52674, row 11, _idx 445 772, _val 26221 0, volume 17654570
Error: 155.8 (want -1881.1, got -2036.9), file 12185, row 9, _idx 455 774, _val 6376 0, volume 17233509
Error: 225.5 (want 785.7, got 560.1), file 94243, row 17, _idx 105 331, _val 60917 0, volume 11732191
Error: -189.9 (want 280.3, got 470.2), file 92768, row 2, _idx 184 315, _val

Error: 197.9 (want -894.8, got -1092.8), file 37064, row 0, _idx 332 593, _val 26331 0, volume 14246928
Error: 205.1 (want 1023.6, got 818.5), file 90368, row 23, _idx 85 295, _val 24268 0, volume 10105259
Error: 202.3 (want -1419.4, got -1621.7), file 97243, row 51, _idx 398 691, _val 18343 0, volume 15597142
Error: 196.9 (want -1638.4, got -1835.4), file 9229, row 39, _idx 426 733, _val 13184 0, volume 16476003
Error: 281.3 (want 529.9, got 248.6), file 67157, row 3, _idx 142 376, _val 65535 0, volume 11945785
Error: -191.1 (want -1037.2, got -846.1), file 13420, row 22, _idx 361 515, _val 0 0, volume 6613
Error: -323.3 (want -521.3, got -198.0), file 18115, row 22, _idx 289 391, _val 0 0, volume 551
Error: 173.1 (want 1364.7, got 1191.6), file 74389, row 37, _idx 40 245, _val 15462 0, volume 10042001
Error: 173.0 (want -655.9, got -829.0), file 82740, row 22, _idx 283 547, _val 15877 0, volume 14446410
Error: -193.0 (want -268.3, got -75.2), file 93870, row 45, _idx 256 391, _val 0 

Error: 170.7 (want -658.3, got -829.0), file 41464, row 22, _idx 274 547, _val 10254 0, volume 14589538
Error: 161.4 (want -295.2, got -456.6), file 77162, row 21, _idx 238 485, _val 12584 0, volume 13254887
Error: 254.3 (want -911.1, got -1165.5), file 57676, row 5, _idx 335 606, _val 65535 0, volume 14542018
Error: -188.2 (want -510.4, got -322.2), file 79045, row 49, _idx 287 431, _val 0 0, volume 2827
Error: 292.2 (want -612.4, got -904.6), file 27378, row 52, _idx 286 560, _val 65535 0, volume 14683686
Error: 159.9 (want 762.5, got 602.6), file 35277, row 5, _idx 101 325, _val 10321 0, volume 10961824
Error: 277.6 (want 1140.1, got 862.4), file 83623, row 9, _idx 72 289, _val 65535 0, volume 10266126
Error: 197.9 (want -542.8, got -740.7), file 52652, row 27, _idx 263 532, _val 21700 0, volume 14341186
Error: 207.0 (want 895.3, got 688.3), file 6959, row 16, _idx 91 313, _val 28427 0, volume 11160712
Error: 163.2 (want 723.3, got 560.1), file 55885, row 30, _idx 105 331, _val 1149

Error: 216.2 (want -405.0, got -621.2), file 86251, row 16, _idx 277 512, _val 34462 0, volume 12320536
Error: 166.9 (want 1014.7, got 847.8), file 101, row 17, _idx 77 291, _val 14518 0, volume 10831521
Error: 172.4 (want -2040.4, got -2212.8), file 71130, row 0, _idx 478 811, _val 11225 0, volume 18951501
Error: 160.5 (want 1185.9, got 1025.4), file 48272, row 30, _idx 64 267, _val 16363 0, volume 10418716
Error: 245.2 (want 1049.2, got 803.9), file 86887, row 49, _idx 86 297, _val 65535 0, volume 9796143
Error: -189.6 (want -511.8, got -322.2), file 21275, row 35, _idx 287 431, _val 0 0, volume 1170
Error: 208.3 (want -1607.0, got -1815.3), file 4995, row 46, _idx 423 729, _val 20093 0, volume 15502686
Error: 225.7 (want -162.7, got -388.4), file 64562, row 30, _idx 218 474, _val 36756 0, volume 13522139
Error: 203.5 (want -1905.4, got -2108.9), file 27001, row 45, _idx 457 789, _val 21770 0, volume 18869730
Error: 182.7 (want 1178.2, got 995.5), file 53619, row 43, _idx 57 271, _va

Error: 153.3 (want -539.8, got -693.2), file 29803, row 11, _idx 260 524, _val 7354 0, volume 13622746
Error: 223.4 (want 337.5, got 114.1), file 10797, row 18, _idx 165 396, _val 60334 0, volume 12180988
Error: 197.8 (want -465.4, got -663.3), file 83939, row 35, _idx 279 519, _val 15179 0, volume 12192571
Error: -187.2 (want 806.1, got 993.3), file 71675, row 7, _idx 122 244, _val 0 0, volume 2712
Error: 169.4 (want 55.3, got -114.1), file 75695, row 44, _idx 261 397, _val 0 0, volume 144
Error: 223.6 (want 62.8, got -160.7), file 22611, row 0, _idx 189 438, _val 37642 0, volume 12799215
Error: 179.1 (want 327.2, got 148.1), file 62493, row 25, _idx 227 358, _val 0 0, volume 269
Error: 165.6 (want -14.4, got -180.0), file 19621, row 0, _idx 192 441, _val 10914 0, volume 12900483
Error: 207.2 (want -563.1, got -770.3), file 14110, row 53, _idx 278 537, _val 18607 0, volume 13255852
Error: 155.4 (want 1331.8, got 1176.4), file 58432, row 14, _idx 40 247, _val 9804 0, volume 9987893
Err

# Point test

In [21]:
df2 = pd.DataFrame({
    "row":[0],
    "rising_idx":[157],
    "falling_idx":[341],
    "volume":[12039],
    "rising_weight":[35.2498],
    "falling_weight":[-28.1037], 
    "first_val":[12],
    "last_val":[0]})
print(predict(df2, rising_model_polyfit_2, falling_model_polyfit_2))
print(predict(df2, rising_model_polyfit_3, falling_model_polyfit_3))
print(predict(df2, rising_model_polyfit_4, falling_model_polyfit_4))
print('True label = %f' % 509.093)

535.3581952173173
527.4340484429126
529.7267502097624
True label = 509.093000


In [ ]:
import numpy.polynomial.polynomial as poly
ffit = poly.Polynomial(rising_model[::-1])    # instead of np.poly1d

# fig1 = plt.figure()                                                                                           
# ax1 = fig1.add_subplot(111)                                                                                   
# ax1.scatter(train_dataset['rising_idx'] / scale, train_labels)    
# s = np.array(sorted(train_dataset['rising_idx'] / scale)).astype(float)
# plt.plot(s, ffit(s))
# plt.show()